In [ ]:
# dduncan 16/1/18
# running some tests to examine era5 iwp variability on smaller timescales,
#  trying to crack what's going on with day/night differences on annual timescale

from netCDF4 import Dataset
import glob

In [ ]:
ecdir = '/home/dudavid/Dendrite/Dendrite/UserAreas/Dave/EC/'
yr = '15' # year 20x
ts = 3 # time step in hrs of data (cant be changed--downloaded this way)
mons = sorted(glob.glob(ecdir+yr+'*/')) # the vertical profile data are kept in separate folders
print(len(mons))
nm = 2 # can run 12 months but it takes up 20GB+ RAM...
ndays = 31*nm
nt = 8*ndays*1
print(nt)

In [ ]:
gsize = 0.5 # in degrees, normal grid (downloaded this way. if wanting to change output grid, do elsewhere)
# downloaded at much coarser grid for comparison to DARDAR
latmax= 90  # NS latitude limits of grid (downloaded this way)
nx, ny = int(360.0/gsize), int(2.0*latmax/gsize)+1 
# EC has points AT lat/lon intersections, so EC data actually -180 to 179.5, -90 to 90
#mean_grid_iwc = np.zeros([nz, ny, nnx])
save_swp = np.zeros([nt, ny, nx])
count = 0 # initialize
ndays = 0
#iwpall = []

In [ ]:
t1=0
for m in mons[0:nm]:
    mo = m[len(m)-3:len(m)-1]
    nd = int(len(sorted(os.listdir(m)))) # all daily files in the month:
    print('starting mon: ',mo)
    for d in range(nd):#[0:1]:
        da = str(d+1).rjust(2,'0')
        #for fi in range(8):
        file = 'era5.90NS.'+da+'.'+mo+'.20'+yr+'.0.5deg.3hrly.nc'
        f = Dataset(m+file,'r')
        #iwc, swc = np.array(f['ciwc']), np.array(f['cswc'])  #"specific ice/snow water content" in kg/kg
        swp = f['tcsw'][:]
        swp[swp < 0 ] = 0 #np.nan
        #print(np.shape(swp))
        save_swp[t1:(t1+8),:,:] = swp[:,:,:] # [t,y,x]
        t1+=8
                
        #avg_iw[:,:,fi] += iw[:,:]
        #avg_sw[:,:,fi] += sw[:,:]
        #ndays += 1 # count days processed
print(np.shape(save_swp),np.mean(save_swp))
save_swp = save_swp[0:t1,:,:] # trim to ndays processed
print(np.shape(save_swp),np.mean(save_swp))
            


In [ ]:
# some calculations using large array:
save_swp2 = np.zeros([ny,nx]) #init explicitly    
pct = .99 # where to set NAN values
lv = round(pct*t1)
print(lv,t1-lv,t1)
ttemp = save_swp # to not molest save_swp below?
temp = np.sort(ttemp,axis=0)
temp[lv:t1,:,:] = np.nan
#print(temp[lv-20:t1,84,81])
save_swp2 = temp
np.shape(save_swp2)
#print(info(save_swp))
#print(info(save_swp2))
        
stddev = np.std(save_swp,axis=0)
stddev2 = np.nanstd(save_swp2,axis=0) # now with nans for high values
mean1 = np.mean(save_swp,axis=0) 
mean2 = np.nanmean(save_swp2,axis=0) # now with nans for high values
print(info(stddev))
print(info(stddev2))
f01 = plt.figure(figsize=[14,8])
grdmap((stddev-stddev2),-180,90,0,.5,0)

std_hrly = np.zeros([8,ny,nx])
std_hrly2 = np.zeros([8,ny,nx])
for t in range(8):
    std_hrly[t,:,:] = np.std(save_swp[t::8,:,:],axis=0)
    std_hrly2[t,:,:] = np.nanstd(save_swp2[t::8,:,:],axis=0)
    #grdmap(std_hrly[t,:,:],-180,90,.001,1)
        
    
f000= plt.figure(figsize=[14,8])
grdmap(mean1,-180,90,.001,.6,1,'mean all')
f00 = plt.figure(figsize=[14,8])
grdmap(mean2,-180,90,.001,.6,1,'mean '+str(100*pct)+'%')
#print(np.shape(stddev))
f1 = plt.figure(figsize=[14,8])
grdmap((stddev),-180,90,.001,1,1,'sigma all')
f10= plt.figure(figsize=[14,8])
grdmap((stddev2),-180,90,.001,1,1,'sigma '+str(100*pct)+'%')
f2 = plt.figure(figsize=[14,8])
grdmap((std_hrly[0,:,:]),-180,90,.001,1,1,'hr=0')
f3 = plt.figure(figsize=[14,8])
grdmap((std_hrly2[0,:,:]),-180,90,.001,1,1,'hr=0 (2)')
f4 = plt.figure(figsize=[14,8])
grdmap((std_hrly[4,:,:]),-180,90,.001,1,1,'hr=4')

In [ ]:

f91 = plt.figure(figsize=[14,8])
grdplt(np.flipud(stddev-stddev2),-180,180,-90,90,0,.5,0)
f91.savefig('img/kul.jpg',dpi=500)